# Plot with Plotly

In [161]:
from influxdb import InfluxDBClient
client = InfluxDBClient(host='localhost', port=8086)

client.switch_database('binance_trade')
q='select * from binance_trade.autogen.binance_trade'
bin_trade = pd.DataFrame(client.query(q).get_points())

client.switch_database('bitfinex_trade')
q='select * from bitfinex_trade.autogen.bitfinex_trade'
bitfinex_trade = pd.DataFrame(client.query(q).get_points())

client.switch_database('coinbase_trade')
q='select * from coinbase_trade.autogen.coinbase_trade'
cb_trade = pd.DataFrame(client.query(q).get_points())

client.switch_database('binance_order')
q='select * from binance_order.autogen.binance_order'
binance_order = pd.DataFrame(client.query(q).get_points())

client.switch_database('bitfinex_order')
q='select * from bitfinex_order.autogen.bitfinex_order'
bf_order = pd.DataFrame(client.query(q).get_points())

client.switch_database('coinbase_or')
q='select * from coinbase_or.autogen.coinbase_order'
cb_order = pd.DataFrame(client.query(q).get_points())

## merging order values

In [162]:
binance_order['date'] = binance_order['date'].astype(str)
bf_order['date'] = bf_order['date'].astype(str)
cb_order['date'] = cb_order['date'].astype(str)
binance_order['date'] = binance_order['date'].str[:-5]
bf_order['date'] = bf_order['date'].str[:-2]
cb_order['date'] = cb_order['date'].str[:-2]

In [163]:
df_o = pd.merge(bf_order, cb_order, on=['date'])

In [164]:
df_order = pd.merge(df_o, binance_order, on=['date'])

In [165]:
df_order=df_order[["date","asks_x","bids_x","asks_y","bids_y","asks","bids",]] 

In [166]:
df_order=df_order.rename(columns={"asks_x": "bitfinex_asks","bids_x": "bitfinex_bids",
                                  "asks_y": "coinbase_asks","bids_y": "coinbase_bids",
                                  "asks": "binance_asks","bids": "binance_bids"
                                  })

In [167]:
df_order

,date,bitfinex_asks,bitfinex_bids,coinbase_asks,coinbase_bids,binance_asks,binance_bids
0,1609195964,0.02709,0.02707,0.02710,0.02709,0.027109,0.027109
1,1609195968,0.02709,0.02707,0.02710,0.02709,0.027093,0.027093
2,1609195970,0.02711,0.02708,0.02711,0.02710,0.027093,0.027093
3,1609195987,0.02711,0.02708,0.02710,0.02709,0.027060,0.027060
4,1609195990,0.02711,0.02708,0.02708,0.02706,0.027061,0.027061
...,...,...,...,...,...,...,...
10036,1609239167,0.02731,0.02730,0.02732,0.02731,0.027321,0.027321
10037,1609239231,0.02731,0.02730,0.02732,0.02731,0.027313,0.027313
10038,1609239257,0.02731,0.02730,0.02732,0.02731,0.027302,0.027302
10039,1609239267,0.02731,0.02730,0.02731,0.02730,0.027299,0.027299


## merging trade values

In [168]:
bitfinex_trade=bitfinex_trade.rename(columns={"timestamp": "date"})

In [169]:
df = pd.merge(bin_trade, cb_trade, on=['date'])

In [170]:
df = pd.merge(df, bitfinex_trade, on=['date'])

In [171]:
df_trade=df[["date","price_x","price_y","price"]] 

In [172]:
df_trade=df_trade.rename(columns={"price_x": "binance_trade","price_y": "coinbase_trade","price": "bitfinex_trade"})

In [173]:
df_trade

,date,binance_trade,coinbase_trade,bitfinex_trade
0,1.609196e+09,0.027110,0.02707,0.027093
1,1.609196e+09,0.027096,0.02707,0.027093
2,1.609196e+09,0.027096,0.02707,0.027093
3,1.609196e+09,0.027076,0.02707,0.027093
4,1.609196e+09,0.027068,0.02707,0.027093
...,...,...,...,...
10039,1.609239e+09,0.027323,0.02725,0.027305
10040,1.609239e+09,0.027323,0.02725,0.027305
10041,1.609239e+09,0.027304,0.02725,0.027305
10042,1.609239e+09,0.027300,0.02725,0.027305


In [174]:
df_order=df_order.sort_values(by=['date'])
df_trade=df_trade.sort_values(by=['date'])

## Plotting trades over time

In [175]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_trade.date*1000, y=df_trade.binance_trade,
                    mode='lines',
                    name='Binance'))
fig.add_trace(go.Scatter(x=df_trade.date*1000, y=df_trade.coinbase_trade,
                    mode='lines',
                    name='Coinbase'))
fig.add_trace(go.Scatter(x=df_trade.date*1000, y=df_trade.bitfinex_trade,
                    mode='lines', name='Bitfinex'))


fig.update_layout(
    xaxis=dict(
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()

## Plotting asks over time

In [177]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=pd.to_numeric(df_order.date)*1000, y=df_order.binance_asks,
                    mode='lines',
                    name='Binance'))
fig.add_trace(go.Scatter(x=pd.to_numeric(df_order.date)*1000, y=df_order.coinbase_asks,
                    mode='lines',
                    name='Coinbase'))
fig.add_trace(go.Scatter(x=pd.to_numeric(df_order.date)*1000, y=df_order.bitfinex_asks,
                    mode='lines', name='Bitfinex'))

fig.update_layout(
    xaxis=dict(
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()